# DB 연결 및 데이터 테스트

이 노트북은 Flask 애플리케이션의 데이터베이스에 연결하고 `RealEstateTransaction` 및 `PublicParking` 모델에서 데이터를 가져옵니다.

In [1]:
from utils import setup_db_context
import pandas as pd
import numpy as np
from sqlalchemy import select

# DB 컨텍스트 설정
app, db = setup_db_context()

from myapp.models import RealEstateTransaction, PublicParking

데이터베이스 연결 및 앱 컨텍스트 푸시 완료.


## 1. 부동산 실거래가 (Real Estate Transactions)

In [2]:
# SQLAlchemy를 사용하여 데이터 가져오기
transactions = RealEstateTransaction.query.limit(5).all()
print(f"가져온 거래 수: {len(transactions)}")

# DataFrame으로 변환
stmt = select(RealEstateTransaction)
df_ret = pd.read_sql(stmt, db.session.connection())

print(f"총 레코드 수: {len(df_ret)}")
df_ret.head()

가져온 거래 수: 5
총 레코드 수: 235671


,ret_id,reception_year,district_code,district_name,legal_dong_code,legal_dong_name,jibun_type,jibun_type_name,main_number,sub_number,...,amount,building_area,land_area,floor,right_type,cancel_date,construction_year,building_use,declaration_type,broker_district_name
0,2,2025,11320,도봉구,10600,방학동,1,대지,0638,0010,...,14800,37.66,16.0,4,,,1994.0,연립다세대,중개거래,서울 도봉구
1,3,2025,11380,은평구,10800,역촌동,,,,,...,39620,74.08,182.0,,,,1972.0,단독다가구,직거래,
2,4,2025,11500,강서구,10200,등촌동,1,대지,0628,0013,...,10300,26.16,35.71,6,,,1999.0,오피스텔,중개거래,서울 강서구
3,5,2025,11620,관악구,10200,신림동,1,대지,1639,0051,...,12700,18.70,26.16,8,,,2014.0,오피스텔,중개거래,서울 관악구
4,6,2025,11530,구로구,10600,고척동,1,대지,0339,0000,...,84000,59.89,0.0,9,,,2009.0,아파트,중개거래,"서울 구로구, 서울 양천구"


## 조회 기능

In [9]:
max_amount = 100000
district_name = '도봉구'
legal_dong_name = '창동'
building_use = '아파트'

conditions = []
df_ret_filtered = df_ret.copy()
if district_name:
    df_ret_filtered = df_ret_filtered[df_ret_filtered['district_name'] == district_name]
if legal_dong_name:
    df_ret_filtered = df_ret_filtered[df_ret_filtered['legal_dong_name'] == legal_dong_name]
if max_amount is not None:
    df_ret_filtered = df_ret_filtered[df_ret_filtered['amount'] <= max_amount]
if building_use:
    df_ret_filtered = df_ret_filtered[df_ret_filtered['building_use'] == building_use]

df_ret_filtered

,ret_id,reception_year,district_code,district_name,legal_dong_code,legal_dong_name,jibun_type,jibun_type_name,main_number,sub_number,...,amount,building_area,land_area,floor,right_type,cancel_date,construction_year,building_use,declaration_type,broker_district_name
777,779,2025,11320,도봉구,10700,창동,1,대지,0373,0000,...,30200,36.16,0.0,2,,,1991.0,아파트,중개거래,서울 도봉구
790,792,2025,11320,도봉구,10700,창동,1,대지,0038,0000,...,34500,36.16,0.0,3,,,1989.0,아파트,직거래,
1250,1252,2025,11320,도봉구,10700,창동,1,대지,0347,0000,...,67800,79.07,0.0,3,,20250103,1990.0,아파트,중개거래,서울 노원구
1251,1253,2025,11320,도봉구,10700,창동,1,대지,0347,0000,...,67800,79.07,0.0,3,,,1990.0,아파트,중개거래,서울 노원구
1294,1296,2025,11320,도봉구,10700,창동,1,대지,0347,0000,...,52000,49.94,0.0,15,,,1990.0,아파트,중개거래,서울 도봉구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232973,232975,2022,11320,도봉구,10700,창동,1,대지,0347,0000,...,75500,49.94,0.0,11,,,1990.0,아파트,중개거래,서울 도봉구
233084,233086,2022,11320,도봉구,10700,창동,1,대지,0305,0000,...,67500,59.39,0.0,13,,,1990.0,아파트,중개거래,서울 도봉구
234069,234071,2022,11320,도봉구,10700,창동,1,대지,0804,0000,...,66500,59.86,0.0,9,,,1995.0,아파트,중개거래,서울 도봉구
234208,234210,2022,11320,도봉구,10700,창동,1,대지,0807,0000,...,81000,59.88,0.0,21,,,1997.0,아파트,중개거래,서울 도봉구


### 